In [11]:
# title: Decorator sample
# change log: 2021-08-22 add memoization

# package:

# reference:
# https://lin16.github.io/p/python-decorator/

In [12]:
import time

def sum(a, b):
    # 開始測量
    start = time.localtime()
    print(time.strftime("%Y-%m-%d %H:%M:%S", start))

    # 開始執行
    print("開始執行 sum")
    result= a + b

    # 輸出結果
    print("執行時間：%f 秒" % (time.time() - time.mktime(start)))
    return result

print('sum 執行結果', sum(1, 2))


2024-09-25 22:55:05
開始執行 sum
執行時間：0.572237 秒
sum 執行結果 3


In [13]:
import time
def sum(a, b):
    return a + b

def debug(func):
    def wrapper(*args, **kwargs):
        # 開始測量
        start = time.localtime()
        print(time.strftime("%Y-%m-%d %H:%M:%S", start))

        print('開始執行 ', func.__name__)
        result = func(*args, **kwargs)

        # 輸出結果
        print("執行時間：%f 秒" % (time.time() - time.mktime(start)))
        print(func.__name__, '執行結果', result)
    return wrapper

# 可以簡化寫法
# debug(sum)(1, 2)
debug_sum = debug(sum)
debug_sum(1, 2)


2024-09-25 22:55:05
開始執行  sum
執行時間：0.587379 秒
sum 執行結果 3


In [14]:
import time
from functools import wraps

def debug(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # 開始測量
        start = time.localtime()
        print(time.strftime("%Y-%m-%d %H:%M:%S", start))
        print(f"args type: {type(args)}")
        print(f"args: {args}")
        print(f"kwargs type: {type(kwargs)}")
        print(f"kwargs: {kwargs}")
        print('開始執行 ', func.__name__)
        result = func(*args, **kwargs)

        # 輸出結果
        print("執行時間：%f 秒" % (time.time() - time.mktime(start)))
        print(f"執行時間：{(time.time() - time.mktime(start)):f} 秒")
        print(func.__name__, '執行結果', result)
    return wrapper

@debug
def sum(a, b):
    return a + b

sum(1, 2)

# 不加 @wraps 會發生以下問題
# print('func', sum.__name__)
# print('doc', sum.__doc__)
# func名稱變成wrapper
# func wrapper
# doc None

2024-09-25 22:55:05
args type: <class 'tuple'>
args: (1, 2)
kwargs type: <class 'dict'>
kwargs: {}
開始執行  sum
執行時間：0.603364 秒
執行時間：0.603364 秒
sum 執行結果 3


In [15]:
import time
from functools import wraps
import inspect

def debug(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # 取得所有參數名稱
        params = func.__code__.co_varnames[:func.__code__.co_argcount]
        params_list = list(params)
        print(params_list)
        # 取得參數Index
        index = params_list.index("c")
        print(index)
        # 根據Index取值
        #print(args[index])

        # 取參數預設值
        signature_list ={
        k: v.default
        for k, v in inspect.signature(func).parameters.items()
        if v.default is not inspect.Parameter.empty
        }
        print(signature_list)
        signature = inspect.signature(func).parameters["c"].default
        print(signature)

        spec = inspect.getfullargspec(func)
        print(f"spec: {spec}")


        print(f"args type: {type(args)}")
        print(f"args: {args}")
        print(f"kwargs type: {type(kwargs)}")
        print(f"kwargs: {kwargs}")
        print('開始執行 ', func.__name__)
        result = func(*args, **kwargs)

    return wrapper

@debug
def sum(a, b, c: bool=False):
    return a + b

sum(1, 2)

['a', 'b', 'c']
2
{'c': False}
False
spec: FullArgSpec(args=['a', 'b', 'c'], varargs=None, varkw=None, defaults=(False,), kwonlyargs=[], kwonlydefaults=None, annotations={'c': <class 'bool'>})
2024-09-25 22:55:05
args type: <class 'tuple'>
args: (1, 2)
kwargs type: <class 'dict'>
kwargs: {}
開始執行  sum
執行時間：0.622081 秒
sum 執行結果 3


In [23]:
import time
from functools import wraps
import inspect

def get_default_args(func):
    signature = inspect.signature(func)
    return {
        k: v.default
        for k, v in signature.parameters.items()
        if v.default is not inspect.Parameter.empty
    }

def debug(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        kwds = get_default_args(func)
        print('kwds:', kwds)
        kwds.update(kwargs)
        print('kwds:', kwds)

        print(f"args type: {type(args)}")
        print(f"args: {args}")
        print(f"kwargs type: {type(kwargs)}")
        print(f"kwargs: {kwargs}")
        print('開始執行 ', func.__name__)
        result = func(*args, **kwargs)

    return wrapper

@debug
def sum(a, b, c: bool=False):
    return a + b

sum(1, 2, True)

kwds: {'c': False}
kwds: {'c': False}
2024-09-25 22:59:37
args type: <class 'tuple'>
args: (1, 2, True)
kwargs type: <class 'dict'>
kwargs: {}
開始執行  sum
執行時間：0.945893 秒
sum 執行結果 3


In [31]:
import time
from functools import wraps
import inspect

def debug(func):

    @wraps(func)
    def wrapper(*args, **kwargs):
        sig = inspect.signature(func)
        bound = sig.bind(*args, **kwargs)
        bound.apply_defaults()
        print('Calling decorated function')
        print('called with:', bound.arguments)

        print(f"args type: {type(args)}")
        print(f"args: {args}")
        print(f"kwargs type: {type(kwargs)}")
        print(f"kwargs: {kwargs}")
        print('開始執行 ', func.__name__)
        result = func(*args, **kwargs)

    return wrapper

@debug
def sum(a, b, c: bool=False):
    return a + b

sum(1, 2, True)

Calling decorated function
called with: {'a': 1, 'b': 2, 'c': True}
args type: <class 'tuple'>
args: (1, 2, True)
kwargs type: <class 'dict'>
kwargs: {}
開始執行  sum
